# 📋Table of Contents
* INtroduction
* 📚 Import Libraries
* 📖 Read Dataset
* 😅😅Missing Values
* 🖇🖇📭Data Pipeline
* ⚙️Finding and Training Models With Best Hyperparameters
* 👀👁FInal MOdel WIth VOting
* 📊Generate Predictions and Create Submission File

# INtroduction

**Hi , you clicked this notebook, so without any wasting time, let's get started, but some context, this is multiclass prediction model, which has predict the probabilities of the classes of the columns, and submit them into csv file. Hope you will play with it , plz don't forget mention this notebook , if you 're building the model on the top of this notebook**

# 📚Import Libraries 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import optuna
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss, make_scorer
from sklearn.ensemble import VotingClassifier


warnings.filterwarnings("ignore")


# 📖Read Dataset

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s3e26/train.csv')
main = pd.read_csv('/kaggle/input/cirrhosis-patient-survival-prediction/cirrhosis.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e26/test.csv')

In [3]:
main.rename(columns ={'ID':'id'}, inplace = True)
print(main.shape, train.shape)
train = pd.concat([train, main], axis = 0)

(418, 20) (7905, 20)


In [4]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,8323.0,3764.043133,2369.682885,0.00,1662.50,3743.00,5823.50,7904.00
N_Days,8323.0,2024.528776,1094.968696,41.00,1220.00,1831.00,2689.00,4795.00
Age,8323.0,18381.192359,3686.832308,9598.00,15574.00,18713.00,20684.00,28650.00
Bilirubin,8323.0,2.625940,3.847146,0.30,0.70,1.10,3.00,28.00
Cholesterol,8189.0,351.219074,196.775246,120.00,248.00,299.00,392.00,1775.00
Albumin,8323.0,3.545767,0.350697,1.96,3.35,3.58,3.77,4.64
Copper,8215.0,84.421546,76.327480,4.00,39.00,63.00,102.00,588.00
Alk_Phos,8217.0,1823.044883,1913.388685,289.00,834.00,1181.00,1857.00,13862.40
SGOT,8217.0,114.906530,49.134067,26.35,75.95,108.50,137.95,457.25
Tryglicerides,8187.0,115.662636,53.037660,33.00,84.00,104.00,139.00,598.00


# 😅😅Missing Values

In [5]:
print(train.isnull().sum())
test.isnull().sum()

id                 0
N_Days             0
Drug             106
Age                0
Sex                0
Ascites          106
Hepatomegaly     106
Spiders          106
Edema              0
Bilirubin          0
Cholesterol      134
Albumin            0
Copper           108
Alk_Phos         106
SGOT             106
Tryglicerides    136
Platelets         11
Prothrombin        2
Stage              6
Status             0
dtype: int64


id               0
N_Days           0
Drug             0
Age              0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
dtype: int64

**Wow that's miracle there's no missing values yipee!!**

In [6]:
# One-hot encode the target variable

y = train['Status']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X = train.drop(['id', 'Status'], axis=1)
test = test.drop(['id'], axis=1)

In [7]:
num = X.select_dtypes(include=['int64', 'float64']).columns
col = X.select_dtypes(include=['object']).columns

# 🖇🖇📭Data Pipeline

In [8]:
# Preprocessing for numerical data: imputation and scaling
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Preprocessing for categorical data: imputation and one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num),
        ('cat', categorical_transformer, col)])

In [9]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ⚙️Finding  and Training  Models With Best Hyperparameters

In [10]:
#Best for XBG

# # Define the objective function for Optuna
# def objective(trial):

#     params = {
#         'objective': 'multi:softprob',
#         'booster': 'gbtree',
#         'max_depth': trial.suggest_int('max_depth', 3, 10),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'gamma': trial.suggest_float('gamma', 0, 1),
#     }
#     model = XGBClassifier(**params)
    
#     # Create the full pipeline with the preprocessor
#     pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

#     # Train the model
#     pipeline.fit(X_train, y_train)

#     # Make predictions
#     y_pred_proba = pipeline.predict_proba(X_test)

#     # Calculate log loss
#     loss = log_loss(y_test, y_pred_proba)
#     return loss

# # Perform hyperparameter optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=150)

# # Print the best hyperparameters and corresponding accuracy
# best_trial = study.best_trial
# print(f"Best Trial - Hyperparameters: {best_trial.params}")
# print(f"Best Trial - Accuracy: {best_trial.value}")

# Best Trial - Hyperparameters: {'max_depth': 3, 'learning_rate': 0.06554750793955862, 'n_estimators': 700, 'subsample': 0.879561982113763, 'colsample_bytree': 0.6804124845660782, 'gamma': 0.8869652359644636}
# Best Trial - Accuracy: 0.42398305168359196

model = XGBClassifier( max_depth= 3, 
                       learning_rate= 0.06554750793955862, 
                       n_estimators= 700, 
                       subsample= 0.879561982113763, 
                       colsample_bytree= 0.6804124845660782, 
                       gamma= 0.8869652359644636
                       )

XBG_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Train the model
XBG_pipeline.fit(X, y)

# Make predictions
y_pred_proba = XBG_pipeline.predict_proba(X_test)

# Calculate log loss
loss = log_loss(y_test, y_pred_proba)
print(loss)

0.3118125939165731


In [11]:
# Best for LGBM
# # Define the objective function for Optuna
# def objective(trial):

#     params = {
#             'objective': 'multiclass',
#             'metric': 'softmax',
#             'boosting_type': 'gbdt',
#             'num_leaves': trial.suggest_int('num_leaves', 2, 256),
#             'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
#             'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
#             'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#             'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#             'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
#             'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
#         }
#     model = LGBMClassifier(**params)
    
#     # Create the full pipeline with the preprocessor
#     pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

#     # Train the model
#     pipeline.fit(X_train, y_train)

#     # Make predictions
#     y_pred_proba = pipeline.predict_proba(X_test)

#     # Calculate log loss
#     loss = log_loss(y_test, y_pred_proba)
#     return loss

# # Perform hyperparameter optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=150)

# # Print the best hyperparameters and corresponding accuracy
# best_trial = study.best_trial
# print(f"Best Trial - Hyperparameters: {best_trial.params}")
# print(f"Best Trial - Accuracy: {best_trial.value}")

# Best Trial - Hyperparameters: {'num_leaves': 10, 'learning_rate': 0.09945716175211072, 'n_estimators': 100, 'subsample': 0.638372746835219, 'colsample_bytree': 0.7901846422576978, 'reg_alpha': 0.9399890733575376, 'reg_lambda': 0.39927135467931296}
# Best Trial - Accuracy: 0.4320768674184984

model = LGBMClassifier( num_leaves= 10, 
                        learning_rate= 0.09945716175211072, 
                        n_estimators= 100, 
                        subsample= 0.638372746835219, 
                        colsample_bytree= 0.7901846422576978, 
                        reg_alpha= 0.9399890733575376, 
                        reg_lambda= 0.39927135467931296)


LGBM_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Train the model
LGBM_pipeline.fit(X, y)

# Make predictions
y_pred_proba = LGBM_pipeline.predict_proba(X_test)

# Calculate log loss
loss = log_loss(y_test, y_pred_proba)
print(loss)

0.3460685385130759


In [12]:
# Best for CatboostClassifier
# # Define the objective function for Optuna
# def objective(trial):
#     model_params = {
#         'iterations': trial.suggest_int('iterations', 100, 1000, step=100),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
#         'depth': trial.suggest_int('depth', 3, 10),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
#         'bootstrap_type': 'Bayesian',
#         'verbose': False
#     }
#     # Exclude 'subsample' when using 'bayesian' bootstrap type
#     if model_params['bootstrap_type'] == 'Bayesian':
#         del model_params['subsample']
    
#     model = CatBoostClassifier(**model_params)
    
#     # Create the full pipeline with the preprocessor
#     pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

#     # Train the model
#     pipeline.fit(X_train, y_train)

#     # Make predictions
#     y_pred_proba = pipeline.predict_proba(X_test)

#     # Calculate log loss
#     loss = log_loss(y_test, y_pred_proba)
#     return loss

# # Perform hyperparameter optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=150)

# # Print the best hyperparameters and corresponding accuracy
# best_trial = study.best_trial
# print(f"Best Trial - Hyperparameters: {best_trial.params}")
# print(f"Best Trial - Loss: {best_trial.value}")

# Best Trial - Hyperparameters: {'iterations': 500, 'learning_rate': 0.1872180770901252, 'depth': 3, 'subsample': 0.9148112150472516, 'colsample_bylevel': 0.5012028440449813}
# Best Trial - Loss: 0.4332958910610937

model = CatBoostClassifier(iterations= 500, 
                           learning_rate= 0.1872180770901252, 
                           depth= 3,
                           colsample_bylevel= 0.5012028440449813,
                           bootstrap_type = 'Bayesian',
                            verbose = False)

Cat_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Train the model
Cat_pipeline.fit(X, y)

# Make predictions
y_pred_proba = Cat_pipeline.predict_proba(X_test)

# Calculate log loss
loss = log_loss(y_test, y_pred_proba)
print(loss)

0.3492739794460349


In [13]:
# Best for GradientBoosting
    
#     # Define the objective function for Optuna
# def objective(trial):

#     params = {
#             'loss': 'deviance',
#             'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
#             'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
#             'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#             'max_depth': trial.suggest_int('max_depth', 3, 10),
#         }
    
#     model = GradientBoostingClassifier(**params)
    
#     # Create the full pipeline with the preprocessor
#     pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

#     # Train the model
#     pipeline.fit(X_train, y_train)

#     # Make predictions
#     y_pred_proba = pipeline.predict_proba(X_test)

#     # Calculate log loss
#     loss = log_loss(y_test, y_pred_proba)
#     return loss

# # Perform hyperparameter optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=150)

# # Print the best hyperparameters and corresponding accuracy
# best_trial = study.best_trial

# print(f"Best Trial - Hyperparameters: {best_trial.params}")
# print(f"Best Trial - Accuracy: {best_trial.value}")
# Best Trial - Hyperparameters: {'learning_rate': 0.05885877940951943, 'n_estimators': 300, 'subsample': 0.5470272507158589, 'max_depth': 3}
# Best Trial - Accuracy: 0.42324580434605336

model = GradientBoostingClassifier(learning_rate= 0.05885877940951943, 
                                  n_estimators= 300, 
                                  subsample= 0.5470272507158589, 
                                  max_depth= 3)

Gradient_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Train the model
Gradient_pipeline.fit(X, y)

# Make predictions
y_pred_proba = Gradient_pipeline.predict_proba(X_test)

# Calculate log loss
loss = log_loss(y_test, y_pred_proba)
print(loss)

0.3273342846021688


# 👀👁FInal MOdel WIth VOting

In [14]:
# Create a VotingClassifier with the three XGBoost models
voting = VotingClassifier(estimators=[
    ('Model1', XBG_pipeline),
    ('Model2', Gradient_pipeline),
    ('Model3', LGBM_pipeline),
    ('Model4', Cat_pipeline)
], voting='soft')



voting.fit(X, y)

VotingClassifier(estimators=[('Model1',
                              Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('num',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='median')),
                                                                                                ('scaler',
                                                                                                 StandardScaler())]),
                                                                                Index(['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'Stage'],
      dtype='object...
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'Stage'],
      dtype='object')),
                                                                               ('cat',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='most_frequent')),
                                                                                                ('onehot',
                                                                                                 OneHotEncoder(handle_unknown='ignore'))]),
                                                                                Index(['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema'], dtype='object'))])),
                                              ('model',
                                               <catboost.core.CatBoostClassifier object at 0x7b7fe539bc40>)]))],
                 voting='soft')

# 📊Generate Predictions and Create Submission File

In [15]:
predict = voting.predict_proba(test)

sub = pd.read_csv('/kaggle/input/playground-series-s3e26/sample_submission.csv')

sub[['Status_C', 'Status_CL', 'Status_D']] = predict

sub.to_csv('sample_submission.csv', index = False)

In [16]:
sub.head()

,id,Status_C,Status_CL,Status_D
0,7905,0.519229,0.031170,0.449601
1,7906,0.469913,0.250726,0.279361
2,7907,0.029649,0.014825,0.955526
3,7908,0.952499,0.003781,0.043720
4,7909,0.854073,0.038712,0.107216
